# Capstone Project (Greenness)
### This module is responsible for reading in a LandSat Image and a Neon AOP NDVI product for Konza Prairie (KONZ)     
### The outcome is a high level investation of the greenness correlation between these two products
### July 13th, 2018
### Team Member: Katharyn Duffy (katharyn.woods@gmail.com), Reza Khatami (rezaa.khatami@gmail.com), Vernon Visser (vervis@gmail.com), Youssef Kaddoura (yokaddoura@gmail.com)

In [40]:
import gdal
import h5py
import numpy as np
from math import floor
import os
import matplotlib.pyplot as plt
from osgeo import gdal, gdalconst


In [42]:
NDVI_AOP_tif_directory = r'C:\\Users\\GoGators\\Desktop\\Neon\\GrAS\\Data\\NEON_indices-veg-spectrometer-mosaic\\2017\\FullSite\\D06\\2017_KONZ_2\\L3\\Spectrometer\\VegIndices\\'
LAI_AOP_tif_directory = r'C:\\Users\\GoGators\\Desktop\\Neon\\GrAS\\Data\\NEON_lai-spectrometer-mosaic\\2017\\FullSite\\D06\\2017_KONZ_2\\L3\\Spectrometer\\LAI\\'

In [4]:
Tower1_Easting = 710730
Tower1_Northing = 4330787

#Tower2_Easting = 316807
#Tower2_Northing = 4881510

Min_Distance_Tower1 = 2000000.1
#Min_Distance_Tower2 = 2000000.2
Counter = 0
Index1 = 0
#Index2 = 0

Min_Distance_Easting_Tower1= 0
Min_Distance_Northing_Tower1= 0
#Min_Distance_Easting_Tower2= 0
#Min_Distance_Northing_Tower2= 0

files = os.listdir(NDVI_AOP_tif_directory)
NDVI_files = [i for i in files if i.endswith('.tif')]
for file in NDVI_files:
    Counter = Counter + 1
    filename_split = (file).split("_") 
    Easting = np.int(filename_split[4])
    Northing = np.int(filename_split[5])
#    print(Easting,Northing)
    Distance_from_Tower1 = np.sqrt(np.square(Easting-Tower1_Easting) + np.square(Northing-Tower1_Northing))
#    Distance_from_Tower2 = np.sqrt(np.square(Easting-Tower2_Easting) + np.square(Northing-Tower2_Northing))
#    print(Distance_from_Tower1)
    if (Distance_from_Tower1 < 4000):
        if(((Tower1_Easting-Easting) >=0) and ((Tower1_Northing-Northing)>=0)):
            Min_Distance_Tower1 = Distance_from_Tower1
            Min_Distance_Easting_Tower1 =Easting
            Min_Distance_Northing_Tower1 = Northing
            Index1 = Counter
#    if (Distance_from_Tower2 < 4000):
#        if(((Tower1_Easting-Easting) >=0) and ((Tower1_Northing-Northing)>=0)):
#            Min_Distance_Tower2 = Distance_from_Tower2
#            Min_Distance_Easting_Tower2 =Easting
#            Min_Distance_Northing_Tower2 = Northing
#            Index2 = Counter
#print('Tower One is in ',Min_Distance_Easting_Tower1,Min_Distance_Northing_Tower1, ' Index1 ',Index1)
#print('Tower Two is in ',Min_Distance_Easting_Tower2,Min_Distance_Northing_Tower2, ' Index2 ',Index2)
NDVI_file_of_interest = filename_split[0]+'_'+filename_split[1]+'_'+filename_split[2]+'_'+filename_split[3]+'_'+str(Min_Distance_Easting_Tower1)+'_'+str(Min_Distance_Northing_Tower1)+'_'+filename_split[6]
#print(file_of_interest)
NDVI_filepath = "C:\\Users\\GoGators\\Desktop\\Neon\\GrAS\\Data\\NEON_indices-veg-spectrometer-mosaic\\2017\\FullSite\\D06\\2017_KONZ_2\\L3\\Spectrometer\\VegIndices\\"



rC:\Users\GoGators\Desktop\Neon\GrAS\Data\NEON_indices-veg-spectrometer-mosaic\2017\FullSite\D06\2017_KONZ_2\L3\Spectrometer\VegIndices\NEON_D06_KONZ_DP3_710000_4330000_NDVI.tif


In [31]:
NDVI_tif_filename =os.path.join(NDVI_filepath,NDVI_file_of_interest)


C:\Users\GoGators\Desktop\Neon\GrAS\Data\NEON_indices-veg-spectrometer-mosaic\2017\FullSite\D06\2017_KONZ_2\L3\Spectrometer\VegIndices\NEON_D06_KONZ_DP3_710000_4330000_NDVI.tif


In [32]:
#tif_AOP_filename = r'C:\Users\GoGators\Desktop\Neon\GrAS\Data\NEON_indices-veg-spectrometer-mosaic\2017\FullSite\D06\2017_KONZ_2\L3\Spectrometer\VegIndices\NEON_D06_KONZ_DP3_710000_4330000_NDVI.tif'
tif_AOP_filename = tif_filename

# Reading Corresponding LandSat Image and Preparing it for Resampling

In [33]:
src_LS_filename = r'C:\Users\GoGators\Desktop\Neon\GrAS\Data\LS8-May_to_July_NDVI.tif'


In [34]:
# %load raster2array.py
def raster2array(geotif_file):
    metadata = {}
    dataset = gdal.Open(geotif_file)
    metadata['array_rows'] = dataset.RasterYSize
    metadata['array_cols'] = dataset.RasterXSize
    metadata['bands'] = dataset.RasterCount
    metadata['driver'] = dataset.GetDriver().LongName
    metadata['projection'] = dataset.GetProjection()
    metadata['geotransform'] = dataset.GetGeoTransform()
    
    mapinfo = dataset.GetGeoTransform()
    metadata['pixelWidth'] = mapinfo[1]
    metadata['pixelHeight'] = mapinfo[5]

    xMin = mapinfo[0]
    xMax = mapinfo[0] + dataset.RasterXSize/mapinfo[1]
    yMin = mapinfo[3] + dataset.RasterYSize/mapinfo[5]
    yMax = mapinfo[3]
    
    metadata['extent'] = (xMin,xMax,yMin,yMax)
    
    raster = dataset.GetRasterBand(1)
    array_shape = raster.ReadAsArray(0,0,metadata['array_cols'],metadata['array_rows']).astype(np.float).shape
    metadata['noDataValue'] = raster.GetNoDataValue()
    metadata['scaleFactor'] = raster.GetScale()
    
    array = np.zeros((array_shape[0],array_shape[1],dataset.RasterCount),'uint8') #pre-allocate stackedArray matrix
    
    if metadata['bands'] == 1:
        raster = dataset.GetRasterBand(1)
        metadata['noDataValue'] = raster.GetNoDataValue()
        metadata['scaleFactor'] = raster.GetScale()
              
        array = dataset.GetRasterBand(1).ReadAsArray(0,0,metadata['array_cols'],metadata['array_rows']).astype(np.float)
        array[np.where(array==metadata['noDataValue'])]=np.nan
        array = array/metadata['scaleFactor']
    
    elif metadata['bands'] > 1:    
        for i in range(1, dataset.RasterCount+1):
            band = float(dataset.GetRasterBand(i).ReadAsArray(0,0,metadata['array_cols'],metadata['array_rows']))
            band[np.where(band==metadata['noDataValue'])]=np.nan
            band = band/metadata['scaleFactor']
            array[...,i-1] = band

    return array, metadata

In [35]:
tif_array, tif_array_metadata = raster2array(tif_AOP_filename)

# Resampling for LS to match AOP NDVI Product

In [36]:
src = gdal.Open(src_LS_filename, gdalconst.GA_ReadOnly)
src_proj = src.GetProjection()
src_geotrans = src.GetGeoTransform()

In [75]:
# We want a section of source that matches this:
match_filename = tif_AOP_filename
match_ds = gdal.Open(match_filename, gdalconst.GA_ReadOnly)
match_proj = match_ds.GetProjection()
match_geotrans = match_ds.GetGeoTransform()
wide = match_ds.RasterXSize
high = match_ds.RasterYSize

In [76]:
# Output / destination
dst_filename = '..//Output//LS_Section_Matching_AOP_NDVI.tif'
dst = gdal.GetDriverByName('GTiff').Create(dst_filename, wide, high, 1, gdalconst.GDT_Float32)
dst.SetGeoTransform( match_geotrans )
dst.SetProjection( match_proj)
gdal.ReprojectImage(src, dst, src_proj, match_proj, gdalconst.GRA_Bilinear)
del dst # Flush

# Reading and Identifying LAI

In [78]:
files2 = os.listdir(LAI_AOP_tif_directory)
LAI_files = [i for i in files2 if i.endswith('.tif')]
filename2_split = LAI_files[0].split("_")
LAI_file_of_interest = filename2_split[0]+'_'+filename2_split[1]+'_'+filename2_split[2]+'_'+filename2_split[3]+'_'+str(Min_Distance_Easting_Tower1)+'_'+str(Min_Distance_Northing_Tower1)+'_'+filename2_split[6]
#print(file_of_interest)
LAI_filepath = "C:\\Users\\GoGators\\Desktop\\Neon\\GrAS\\Data\\NEON_lai-spectrometer-mosaic\\2017\\FullSite\\D06\\2017_KONZ_2\\L3\\Spectrometer\\LAI\\"
LAI_tif_filename =os.path.join(LAI_filepath,LAI_file_of_interest)
print(LAI_tif_filename)



C:\Users\GoGators\Desktop\Neon\GrAS\Data\NEON_lai-spectrometer-mosaic\2017\FullSite\D06\2017_KONZ_2\L3\Spectrometer\LAI\NEON_D06_KONZ_DP3_710000_4330000_LAI.tif
